In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D,BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10

import numpy as np
import os

In [2]:
batch_size=32
epochs=20
data_augmentation=True
num_classes=10

subtract_pixel_mean=True

In [3]:
n=6
version=2

if version ==1:
    depth =n*6+2
elif version==2:
    depth=n*9+2

model_type=f"ResNet{depth}v{version}"


In [4]:
model_type

'ResNet56v2'

In [5]:
(x_train,y_train), (x_test,y_test)=cifar10.load_data()
input_shape=x_train.shape
print(input_shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 744s 4us/step
(50000, 32, 32, 3)


In [6]:
input_shape=x_train.shape[1:]
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

if subtract_pixel_mean:
    x_train_mean=np.mean(x_train, axis=0)
    x_train-=x_train_mean
    x_test-=x_train_mean

print("x_train shape:", x_train.shape)
print(x_train.shape[0],"train samples")
print(x_test.shape[0],"test samples")
print("y_train shape:",y_train.shape)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [7]:
#One hot encoding
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [8]:
def lr_schedule(epoch):
    lr=1e-3
    if epoch >= 80:
        lr *= 0.5e-3
    elif epoch >= 60:
        lr *= 1e-3
    elif epoch >= 40:
        lr *= 1e-2
    elif epoch >= 20:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [9]:
def resnet_layer(
        inputs,
        num_filters=16,
        kernel_size=3,
        strides=1,
        activation='relu',
        batch_normalization=True,
        conv_first=True):
    conv=Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding="same",
                kernel_initializer='he_normal', kernel_regularizer=l2(1e-4))
    x=inputs
    if conv_first:
        x=conv(x)
        if batch_normalization:
            x=BatchNormalization()(x)
        if activation is not None:
            x=Activation(activation)(x)
    else:
        if batch_normalization:
            x=BatchNormalization()(x)
        if activation is not None:
            x=Activation(activation)(x)
        x=conv(x)
    return x


In [10]:
def resnet_v1(input_shape,depth,num_classes=10):
    if (depth - 2) % 6!= 0:
        raise ValueError("depth should be 6n+2 (where n is a non-negative integer).")
    num_filters = 16
    num_blocks = int((depth - 2) / 6)
    inputs=Input(shape=input_shape)
    x=resnet_layer(inputs=inputs)

    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides=1
            if stack > 0 and res_block == 0:
                strides=2
            y=resnet_layer(inputs=x, num_filters=num_filters, strides=strides)
            y=resnet_layer(inputs=y, num_filters=num_filters, activation=None)
            if stack>0 and res_block == 0:
                x=resnet_layer(inputs=x, num_filters=num_filters,kernel_size=1
                               ,strides=strides,activation=None,
                               batch_normalization=False)
            
            x=keras.layers.add([x,y])
            x=Activation('relu')(x)
        num_filters*=2
    x=AveragePooling2D(pool_size=8)(x)
    y=Flatten()(x)
    outputs=Dense(num_classes,
                  Activation='softmax',
                  kernel_initializer='he_normal')(y)
    model=Model(inputs=inputs, outputs=outputs)
    return model